In [1]:
from numpy import array

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [3]:
data=""""Orientation - terms and concepts in system development
The purpose of the sub-part is to orientate oneself generally among different different terms and concepts in system development and does not need to be overworked in any way, everyone who has participated and collaborated with their group will be approved in this part. Those who can briefly and in their own words explain a complicated subject in a simple and understandable language are VG candidates. However, it is the ability to analyze and object model the task that will be decisive for the final grade in Introduction to programming.

Team Database: This group delves into databases, each member chooses a database to delve into and writes a summary of the database's most important features, e.g. how to download installs and configures, it is possible to.comply within the group with the same database. The group compiles a document and reports the results."""

In [4]:

# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

In [5]:
vocab_size=len(tokenizer.word_index)+1
print(vocab_size)

96


In [6]:
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)


Vocabulary Size: 96


In [7]:
# create word -> word sequences
sequences = list()
for i in range(1, len(encoded)):
	sequence = encoded[i-1:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 151


In [8]:
# split into X and y elements
sequences = array(sequences)
X, y = sequences[:,0],sequences[:,1]

In [9]:
X[:5],y[:5]

(array([24, 10,  1, 11,  3]), array([10,  1, 11,  3, 12]))

In [10]:
# one hot encode outputs
y= to_categorical(y, num_classes=vocab_size)
y[:5]
            

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [11]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 10)             960       
_________________________________________________________________
lstm (LSTM)                  (None, 50)                12200     
_________________________________________________________________
dense (Dense)                (None, 96)                4896      
Total params: 18,056
Trainable params: 18,056
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
# compile network
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
#model.fit(X, y, epochs=500, verbose=2)

In [27]:
# generate a sequence from the model
def generate_seq(model, tokenizer, seed_text, n_words):
	in_text, result = seed_text, seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		encoded = array(encoded)
		# predict a word in the vocabulary
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text, result = out_word, result + '  ' + out_word
	return result

In [32]:
print(generate_seq(model, tokenizer, "  oneself ", 6))

  oneself   document  those  g  that  delve  download
